<a href="https://colab.research.google.com/github/gdhanush27/Deepfake-audio-detection/blob/main/Sem7Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Download Data
# @markdown ---
# @markdown > Download dataset from github
# @markdown ---
!git clone https://github.com/gdhanush27/Deepfake-audio-detection.git
from google.colab import output
output.clear()

In [ ]:
# @title Load data
# @markdown ---
# @markdown > Load data
# @markdown ---

img_size_ = 32 # @param [24,32,64,128] {type:"raw"}

import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, concatenate
import joblib
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import google
import numpy as np

save_path = f'/content/Deepfake-audio-detection/DataFiles/dataset_{img_size_}_{img_size_}.pkl'
try:
  data = joblib.load(save_path)
except FileNotFoundError:
  print("Data Not Found")
else:
  cnn_images = data['cnn_images']
  rnn_images = data['rnn_images']
  labels = data['labels']
  X_cnn_train, X_cnn_val, X_rnn_train, X_rnn_val, y_train, y_val = train_test_split(
    cnn_images, rnn_images, labels, test_size=0.2, random_state=42)
  class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
  class_weight_dict = dict(enumerate(class_weights))
  print(f"Data Loaded Successfully\nData : /content/drive/MyDrive/dataset_{img_size_}_{img_size_}.pkl")

Data Loaded Successfully
Data : /content/drive/MyDrive/dataset_32_32.pkl


In [ ]:
# @markdown ---
# @markdown > Only Run this, If data not found...
# @markdown ---
# @title Data Preprocessing *ONLY IF NECESSARY*

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, TimeDistributed, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.model_selection import train_test_split
import joblib

prepared_dir = "/content/drive/MyDrive/DatasetUpdated"

cnn_dir = os.path.join(prepared_dir, "CNN")
rnn_dir = os.path.join(prepared_dir, "RNN")

def load_data(cnn_dir, rnn_dir, max_rnn_images=100):
    cnn_images = []
    rnn_images = []
    labels = []
    c = 1
    for class_type in ["REAL", "FAKE"]:
        cnn_class_dir = os.path.join(cnn_dir, class_type)
        rnn_class_dir = os.path.join(rnn_dir, class_type)

        for cnn_file in os.listdir(cnn_class_dir):
            cnn_path = os.path.join(cnn_class_dir, cnn_file)
            rnn_subdir = os.path.join(rnn_class_dir, cnn_file.split(".")[0])

            if os.path.exists(rnn_subdir):
                # Load CNN image
                cnn_img = tf.keras.preprocessing.image.load_img(cnn_path, target_size=(img_size_, img_size_))
                cnn_img = tf.keras.preprocessing.image.img_to_array(cnn_img) / 255.0

                # Load RNN images (each segment)
                rnn_imgs = []
                for rnn_file in os.listdir(rnn_subdir):
                    rnn_path = os.path.join(rnn_subdir, rnn_file)
                    rnn_img = tf.keras.preprocessing.image.load_img(rnn_path, target_size=(img_size_, img_size_))
                    rnn_img = tf.keras.preprocessing.image.img_to_array(rnn_img) / 255.0
                    rnn_imgs.append(rnn_img)

                # Pad or truncate the RNN images to ensure equal length
                if len(rnn_imgs) < max_rnn_images:
                    rnn_imgs.extend([np.zeros_like(rnn_imgs[0])] * (max_rnn_images - len(rnn_imgs)))
                else:
                    rnn_imgs = rnn_imgs[:max_rnn_images]

                # Append to lists
                cnn_images.append(cnn_img)
                rnn_images.append(rnn_imgs)
                labels.append(0 if class_type == "REAL" else 1)
                if c%10==0:
                  print(c,"Completed")
                c+=1
    # Convert to numpy arrays
    cnn_images = np.array(cnn_images)
    rnn_images = np.array(rnn_images)
    labels = np.array(labels)

    return cnn_images, np.array(rnn_images), labels

# Load the data
cnn_images, rnn_images, labels = load_data(cnn_dir, rnn_dir)

save_path = f'/content/drive/MyDrive/dataset_{img_size_}_{img_size_}.pkl'
# print(save_path)
data = {
    'cnn_images': cnn_images,
    'rnn_images': rnn_images,
    'labels': labels
}

joblib.dump(data, save_path)
print(f"Data saved to {save_path}")

Data saved to /content/drive/MyDrive/dataset_64_64.pkl


In [ ]:
# @title Compression *ONLY IF NECESSARY*
# @markdown ---
# @markdown Compression level (from 1 to 9, with 9 being maximum compression but slower)
# @markdown
# @markdown ---
comp_lvl = 2 # @param {type:"slider", min:1, max:10, step:1}
save_path = f'/content/drive/MyDrive/dataset_{img_size_}_{img_size_}_compression_level_{comp_lvl}.pkl'
data = {
    'cnn_images': cnn_images,
    'rnn_images': rnn_images,
    'labels': labels
}
joblib.dump(data, save_path, compress=('zlib', comp_lvl))
print(f"Compresssed to {save_path}")

Compresssed to /content/drive/MyDrive/dataset_128_128_compression_level_3.pkl


In [ ]:
# @title Model Training
# @markdown ---
# @markdown > Train the model
# @markdown ---

num_epochs_ = 1 # @param [1,5,10,25,100] {"type":"raw","allow-input":true}
batch_size_ = 64 # @param ["2","4","8","16","32","64"] {"type":"raw","allow-input":true}

cnn_input_shape = (img_size_, img_size_, 3)
rnn_input_shape = (100, img_size_, img_size_, 3)

cnn_input = Input(shape=cnn_input_shape)
x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
x = MaxPooling2D((2, 2))(x)
# x = Conv2D(64, (3, 3), activation='relu')(x)
# x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
cnn_out = Flatten()(x)

rnn_input = Input(shape=rnn_input_shape)
y = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(rnn_input)
# y = TimeDistributed(MaxPooling2D((2, 2)))(y)
# y = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(y)
y = TimeDistributed(MaxPooling2D((2, 2)))(y)
y = TimeDistributed(Flatten())(y)
rnn_out = LSTM(128)(y)

combined = concatenate([cnn_out, rnn_out])
z = Dense(64, activation='relu')(combined)
z = Dense(16, activation='relu')(z)
output = Dense(1, activation='sigmoid')(z)
model = Model(inputs=[cnn_input, rnn_input], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    [X_cnn_train, X_rnn_train],
    y_train,
    # validation_data=([X_cnn_val, X_rnn_val], y_val),
    epochs=num_epochs_,
    batch_size= batch_size_,
    class_weight=class_weight_dict
)

ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/functional_1/time_distributed_2_1/strided_slice_97/StridedSliceGrad defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/iol

In [ ]:
# @title Evaluate the model
loss, accuracy = model.evaluate([X_cnn_val, X_rnn_val], y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8462 - loss: 0.6854
Validation Loss: 0.6854361295700073
Validation Accuracy: 0.8461538553237915


In [ ]:
# @title Save model
model.save('/content/drive/MyDrive/fusion_model.h5')

In [ ]:
# @title Plot Graph
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()